*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Text Classification of MultiNLI Sentences using BERT with Azure ML Pipelines

In [2]:
import sys
sys.path.append("../../")
import os
import json
import random
import shutil
import pandas as pd

from utils_nlp.bert.common import Language, Tokenizer
from utils_nlp.azureml import azureml_utils
from utils_nlp.dataset.multinli import get_generator

from sklearn.preprocessing import LabelEncoder
from azureml.core import Datastore, Experiment,  get_run
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import ComputeTarget
from azureml.exceptions import ComputeTargetException
from azureml.data.data_reference import DataReference
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.widgets import RunDetails
from azureml.train.dnn import PyTorch
from azureml.core.runconfig import MpiConfiguration
from azureml.pipeline.steps import EstimatorStep

## 0. Introduction

In this notebook, we fine-tune and evaluate a pretrained BERT model on a subset of the MultiNLI dataset by using Azure ML Pipelines.

In [30]:
LABEL_COL = "genre"
TEXT_COL = "sentence1"
DATA_FOLDER = "../../data/temp"
TRAIN_FOLDER = "../../data/temp/train"
TEST_FOLDER = "../../data/temp/test"
BERT_CACHE_DIR = "../../data/temp"
LANGUAGE = Language.ENGLISH
TO_LOWER = True
MAX_LEN = 150
BATCH_SIZE = 32
NUM_GPUS = 2
NUM_EPOCHS = 1
TRAIN_SIZE = 0.6
TEXT_COL = "sentence1"
ENCODED_LABEL_COL = "label"
TOKEN_COL = "tokens"
MASK_COL = "mask"
NUM_BATCHES = None
LABELS = ['telephone', 'government', 'travel', 'slate', 'fiction']
PROJECT_FOLDER = "../.."
TEMP_FOLDER = "temp"

In this example we will use AzureML pipelines to execute our training pipelines. Each preprocessing step is included as a step in the pipeline. For a more detailed walkthrough of what pipelines are with a getting started guidelines check this [notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-getting-started.ipynb). We start by doing some AzureML related setup below

### 0.1 Create a workspace

First, go through the [Configuration](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`. This will create a config.json file containing the values needed below to create a workspace.

**Note**: you do not need to fill in these values if you have a config.json in the same folder as this notebook

In [4]:
ws = azureml_utils.get_or_create_workspace(
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group="<RESOURCE_GROUP>",
    workspace_name="<WORKSPACE_NAME>",
    workspace_region="<WORKSPACE_REGION>",
)

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


### 0.2 Create a compute target

In [ ]:
# choose your cluster
cluster_name = "pipelines-tc-12"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_NC12", max_nodes=8
    )

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute.
print(compute_target.get_status().serialize())

## 1. Preprocessing

The pipeline is defined by a series of steps, the first being a PythonScriptStep which utilizes [DASK](https://dask.org/) to load dataframes in batches allowing us to load and preprocess different sets of data in parallel.

### 1.1 Read Dataset

In [33]:
train_batches = get_generator(DATA_FOLDER, "train", num_batches=NUM_BATCHES, batch_size=10e6)
test_batches = get_generator(DATA_FOLDER, "dev_matched", num_batches=NUM_BATCHES, batch_size=10e6)

### 1.2 Preprocess and Tokenize

In the classification task, use the first sentence only as the text input, and the corresponding genre as the label. Select the examples corresponding to one of the entailment labels (*neutral* in this case) to avoid duplicate rows, as the sentences are not unique, whereas the sentence pairs are.

Once filtered, we encode the labels. To do this, fit a label encoder with the know labels in a MNLI dataset.

In [37]:
os.makedirs(TRAIN_FOLDER, exist_ok=True)
os.makedirs(TEST_FOLDER, exist_ok=True)
os.makedirs(os.path.join(PROJECT_FOLDER, TEMP_FOLDER), exist_ok=True)

labels = LABELS
label_encoder = LabelEncoder()
label_encoder.fit(labels)

num_train_batches = 0
for batch in train_batches:
    batch = batch[batch["gold_label"]=="neutral"]
    batch[ENCODED_LABEL_COL] = label_encoder.transform(batch[LABEL_COL])
    batch.to_csv(TRAIN_FOLDER+"/batch{}.csv".format(str(num_train_batches)))
    num_train_batches += 1
    
num_test_batches = 0
for batch in test_batches:
    batch = batch[batch["gold_label"]=="neutral"]
    batch[ENCODED_LABEL_COL] = label_encoder.transform(batch[LABEL_COL])
    batch.to_csv(TEST_FOLDER+"/batch{}.csv".format(str(num_test_batches)))
    num_test_batches += 1

Once we have batches of data ready they are uploaded to the datastore.

In [9]:
ds = ws.get_default_datastore()
ds.upload(src_dir=TRAIN_FOLDER, target_path="mnli_data/train", overwrite=True, show_progress=False)
ds.upload(src_dir=TEST_FOLDER, target_path="mnli_data/test", overwrite=True, show_progress=False)

$AZUREML_DATAREFERENCE_19a8b227ca85497ba549bf5ba4f6e21d

In [10]:
shutil.rmtree(TRAIN_FOLDER)
shutil.rmtree(TEST_FOLDER)

We can now parallely operate on each batch to tokenize the data and preprocess the tokens. To do this, we create a PythonScript step below.

In [38]:
%%writefile $PROJECT_FOLDER/$TEMP_FOLDER/preprocess.py
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.
import argparse
import logging
import os

import pandas as pd

from utils_nlp.bert.common import Language, Tokenizer

LABEL_COL = "genre"
TEXT_COL = "sentence1"
LANGUAGE = Language.ENGLISH
TO_LOWER = True
MAX_LEN = 150

logger = logging.getLogger(__name__)


def tokenize(df):
    """Tokenize the text documents and convert them to lists of tokens using the BERT tokenizer.
    Args:
        df(pd.Dataframe): Dataframe with training or test samples

    Returns:

        list: List of lists of tokens for train set.

    """
    tokenizer = Tokenizer(
        LANGUAGE, to_lower=TO_LOWER)
    tokens = tokenizer.tokenize(list(df[TEXT_COL]))

    return tokens


def preprocess(tokens):
    """ Preprocess method that does the following,
            Convert the tokens into token indices corresponding to the BERT tokenizer's vocabulary
            Add the special tokens [CLS] and [SEP] to mark the beginning and end of a sentence
            Pad or truncate the token lists to the specified max length
            Return mask lists that indicate paddings' positions
            Return token type id lists that indicate which sentence the tokens belong to (not needed
            for one-sequence classification)

    Args:
        tokens(pd.Dataframe): Dataframe with tokens for train set.

    Returns:
        list: List of lists of tokens for train or test set with special tokens added.
        list: Input mask.
    """
    tokenizer = Tokenizer(
        LANGUAGE, to_lower=TO_LOWER)
    tokens, mask, _ = tokenizer.preprocess_classification_tokens(
        tokens, MAX_LEN
    )

    return tokens, mask


parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str, help="input data")
parser.add_argument("--output_data", type=str, help="Path to the output file.")

args = parser.parse_args()
input_data = args.input_data
output_data = args.output_data
output_dir = os.path.dirname(os.path.abspath(output_data))

if output_dir is not None:
    os.makedirs(output_dir, exist_ok=True)
    logger.info("%s created" % output_dir)

df = pd.read_csv(args.input_data)
tokens_array = tokenize(df)
tokens_array, mask_array = preprocess(tokens_array)

df['tokens'] = tokens_array
df['mask'] = mask_array

# Filter columns
cols = ['tokens', 'mask', 'label']
df = df[cols]
df.to_csv(output_data, header=False, index=False)
logger.info("Completed")

Writing ../../temp/preprocess.py


Create a conda environment for the steps below.

In [12]:
conda_dependencies = CondaDependencies.create(
    conda_packages=[
        "numpy",
        "scikit-learn",
        "pandas",
    ],
    pip_packages=["azureml-sdk==1.0.43.*", 
                  "torch==1.1", 
                  "tqdm==4.31.1",
                 "pytorch-pretrained-bert>=0.6"],
    python_version="3.6.8",
)
run_config = RunConfiguration(conda_dependencies=conda_dependencies)
run_config.environment.docker.enabled = True

Then create the list of steps that use the preprocess.py created above. We use the output of these steps as input to training in the next section.

In [13]:
processed_train_files = []
processed_test_files = []
ds = ws.get_default_datastore()

for i in range(num_train_batches):
        input_data = DataReference(datastore=ds, 
                                   data_reference_name='train_batch_{}'.format(str(i)), 
                                   path_on_datastore='mnli_data/train/batch{}.csv'.format(str(i)),
                                   overwrite=False)

        output_data = PipelineData(name="train{}".format(str(i)), datastore=ds,
                       output_path_on_compute='mnli_data/processed_train/batch{}.csv'.format(str(i)))

        step = PythonScriptStep(
            name='preprocess_step_train_{}'.format(str(i)),
            arguments=["--input_data", input_data, "--output_data", output_data],
            script_name= "utils_nlp/bert/preprocess.py",
            inputs=[input_data],
            outputs=[output_data],
            source_directory=project_folder,
            compute_target=compute_target,
            runconfig=run_config,
            allow_reuse=False,
        )
        
        processed_train_files.append(output_data)         
            
for i in range(num_test_batches):
            input_data = DataReference(datastore=ds, 
                                       data_reference_name='test_batch_{}'.format(str(i)), 
                                       path_on_datastore='mnli_data/test/batch{}.csv'.format(str(i)),
                                       overwrite=False)
        
            output_data = PipelineData(name="test{}".format(str(i)), datastore=ds,
                        output_path_on_compute='mnli_data/processed_test/batch{}.csv'.format(str(i)))
            
            step = PythonScriptStep(
                name='preprocess_step_test_{}'.format(str(i)),
                arguments=["--input_data", input_data, "--output_data", output_data],
                script_name= "utils_nlp/bert/preprocess.py",
                inputs=[input_data],
                outputs=[output_data],
                source_directory=project_folder,
                compute_target=compute_target,
                runconfig=run_config,
                allow_reuse=False,
            )
            
            processed_test_files.append(output_data)

## 2. Train and Score

Once the data is processed and available on datastore, we  train the classifier using the training examples. This involves fine-tuning the BERT Transformer and learning a linear classification layer on top of that. After training is complete we score the performance of the model on the test dataset

The training is distributed and is done AzureML's capability to support distributed using MPI with horovod. 


### 2.1 Setup training script

In [39]:
%%writefile $PROJECT_FOLDER/train.py
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

import argparse
import json
import logging
import os
import torch
from sklearn.metrics import classification_report

from utils_nlp.bert.common import Language
from utils_nlp.bert.sequence_classification_distributed import (
    BERTSequenceDistClassifier,
)
from utils_nlp.common.timer import Timer

BATCH_SIZE = 32
NUM_GPUS = 2
NUM_EPOCHS = 1
LABELS = ["telephone", "government", "travel", "slate", "fiction"]
OUTPUT_DIR = "./outputs/"

logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument(
    "--train_files",
    nargs="+",
    default=[],
    help="List of file paths to all the files in train dataset.",
)

parser.add_argument(
    "--test_files",
    nargs="+",
    default=[],
    help="List of file paths to all the files in test dataset.",
)

args = parser.parse_args()
train_files = [file.strip() for file in args.train_files]
test_files = [file.strip() for file in args.test_files]

# Handle square brackets from train list
train_files[0] = train_files[0][1:]
train_files[len(train_files) - 1] = train_files[len(train_files) - 1][:-1]

# Handle square brackets from test list
test_files[0] = test_files[0][1:]
test_files[len(test_files) - 1] = test_files[len(test_files) - 1][:-1]

# Train
classifier = BERTSequenceDistClassifier(
    language=Language.ENGLISH, num_labels=len(LABELS)
)
with Timer() as t:
    classifier.fit(
        train_files,
        num_gpus=NUM_GPUS,
        num_epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=True,
    )

# Predict
preds, labels_test = classifier.predict(
    test_files, num_gpus=NUM_GPUS, batch_size=BATCH_SIZE
)

results = classification_report(
    labels_test, preds, target_names=LABELS, output_dict=True
)

# Write out results.
result_file = os.path.join(OUTPUT_DIR, "results.json")
with open(result_file, "w+") as fp:
    json.dump(results, fp)

# Save model
model_file = os.path.join(OUTPUT_DIR, "model.pt")
torch.save(classifier.model.state_dict(), model_file)

Overwriting ../../train.py


### 2.2 Create a Pytorch Estimator

We create a Pytorch Estimator using AzureML SDK and additonally define an EstimatorStep to run it on AzureML pipelines.

In [47]:
estimator = PyTorch(source_directory=project_folder,
                    compute_target=compute_target,
                    entry_script='utils_nlp/bert/train.py',
                    node_count=4,
                    distributed_training=MpiConfiguration(),
                    process_count_per_node=2,
                    use_gpu=True,
                    conda_packages=['scikit-learn=0.20.3', 'numpy>=1.16.0', 'pandas'],
                    pip_packages=["tqdm==4.31.1","pytorch-pretrained-bert>=0.6",'azureml-mlflow>=1.0.43.1']
                   )

WARNING - framework_version is not specified, defaulting to version 1.1.
WARNING - 'process_count_per_node' parameter will be deprecated. Please use it as part of 'distributed_training' parameter.


In [48]:
inputs = processed_train_files + processed_test_files

est_step = EstimatorStep(name="Estimator-Train", 
                         estimator=estimator, 
                         estimator_entry_script_arguments=[
                             '--train_files',  str(processed_train_files),
                             '--test_files', str(processed_test_files)],
                         inputs = inputs,
                         runconfig_pipeline_params=None, 
                         compute_target=compute_target)

### 2.3 Submit the pipeline.

In [49]:
pipeline = Pipeline(workspace=ws, steps=[est_step])
experiment = Experiment(ws, 'NLP-TC-BERT-distributed')
pipeline_run = experiment.submit(pipeline)

Created step Estimator-Train [83d1dfb9][1a0e49ec-c2cb-407c-985a-c5ceaf2a69c4], (This step will run and generate new outputs)
Created step preprocess_step_train_0 [632e6788][9b726ae0-d1b9-4165-b816-dc340a4dcce3], (This step will run and generate new outputs)
Created step preprocess_step_train_1 [171b0953][2cb6c894-723e-4839-a6f4-e8826d0bd88b], (This step will run and generate new outputs)
Created step preprocess_step_train_2 [e48a6105][781554a9-df63-4ba9-b82d-ebc2fd24d1bd], (This step will run and generate new outputs)
Created step preprocess_step_train_3 [3b917cd5][dc7fa041-31be-4215-b8a9-c9964e1c5aec], (This step will run and generate new outputs)
Created step preprocess_step_train_4 [2e0457ae][5e0d5960-a316-4871-88c1-5812539e4740], (This step will run and generate new outputs)
Created step preprocess_step_train_5 [69954270][0e28a297-c7fb-42d8-905a-5f8d551e4786], (This step will run and generate new outputs)
Created step preprocess_step_train_6 [969503b9][ec9354ef-44b3-4b11-bd91-21103

In [50]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', '…

In [ ]:
#If you would like to cancel the job for any reasons uncomment the code below.
#experiment = Experiment(ws, name='NLP-TC-BERT-distributed')
#run = get_run(experiment, '2f2eb9b3-f9a1-42cf-a846-057572277a7f')
#run.cancel()

### 2.4 Download and analyze results

In [9]:
step_run = pipeline_run.find_step_run("Estimator-Train")[0]
file_names = ['outputs/results.json', 'outputs/model.pt']
azureml_utils.get_output_files(step_run, './outputs', file_names=file_names)

In [16]:
with open('outputs/results.json', 'r') as handle:
    parsed = json.load(handle)
    print(pd.DataFrame.from_dict(parsed).transpose())

              f1-score  precision    recall  support
telephone     0.922724   0.889381  0.958665    629.0
government    0.960270   0.972603  0.948247    599.0
travel        0.861564   0.916811  0.812596    651.0
slate         0.992713   0.993517  0.991909    618.0
fiction       0.941815   0.912782  0.972756    624.0
micro avg     0.935598   0.935598  0.935598   3121.0
macro avg     0.935817   0.937019  0.936835   3121.0
weighted avg  0.934849   0.936374  0.935598   3121.0


Finally clean up any intermediate files we created.

In [40]:
shutil.rmtree(os.path.join(PROJECT_FOLDER, TEMP_FOLDER))